1: Chia làm làm  cách làm chunk overlap xong tim đoạn 5 đoạn có tổng điểm cao nhất. Chunk sao cho hết câu. Mỗi đoạn dài <128 token 
2: Dùng LLM để tìm ra nhưng thông tin liên quan 

In [ ]:
import pandas as pd 
import re

In [ ]:
import re
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification


def join_top_chunks(input_text, query_text, max_tokens=128, overlap=20, threshold=512, top_k=5, device="cuda"):
    """
    Trả về (DataFrame top-k chunk, đoạn join lại từ top-k chunk).
    """

    # ======================
    # Hàm chia chunk theo câu
    # ======================
    def chunk_text_by_sentence(text):
        tokens = text.strip().split()
        if len(tokens) <= threshold:
            return [text.strip()]

        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        sentence_tokens = [s.split() for s in sentences if s]

        chunks, current_chunk = [], []
        for sent in sentence_tokens:
            if len(current_chunk) + len(sent) > max_tokens:
                chunks.append(" ".join(current_chunk))
                if overlap > 0 and len(current_chunk) > overlap:
                    current_chunk = current_chunk[-overlap:]
                else:
                    current_chunk = []
            current_chunk.extend(sent)

        if current_chunk:
            chunks.append(" ".join(current_chunk))
        return chunks

    # ======================
    # Load model + tokenizer
    # ======================
    model_name = "thanhtantran/Vietnamese_Reranker"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
    model.eval()

    MAX_LENGTH = 512

    # ======================
    # Thực thi
    # ======================
    chunks = chunk_text_by_sentence(input_text)

    # Tạo list query–chunk pairs
    queries = [query_text] * len(chunks)

    # Tokenize
    inputs = tokenizer(
        queries,
        chunks,
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt"
    ).to(device)

    # Inference
    with torch.no_grad():
        outputs = model(**inputs, return_dict=True)
        scores = outputs.logits.view(-1).float().cpu().numpy()

    # Xếp hạng top-k
    top_indices = scores.argsort()[::-1][:min(top_k, len(chunks))]

    top_indices = sorted(top_indices)

    top_chunks = [(chunks[i], float(scores[i])) for i in top_indices]

    # DataFrame
    top_df = pd.DataFrame(top_chunks, columns=["chunk", "score"])

    # Join text
    joined_text = " ".join([chunks[i] for i in top_indices])

    return top_df, joined_text




In [ ]:
import pandas as pd
from tqdm import tqdm

results = []
data = pd.read_csv("/kaggle/input/hcmus-foursight-vihallu/Vihallu/vihallu-public-test.csv")
for idx, row in tqdm(data.iterrows(), total=len(data), desc="🔄 Processing rows"):
    input_text = row["context"]
    query_text = row["prompt"]
    response_text = row["response"]
    label = row["predict_label"] # Nếu là public thì bỏ 

    try:
        top_df, joined_text = join_top_chunks(input_text, query_text, device="cuda")

        results.append({
            "id": row["id"],              # giữ id gốc
            "context": joined_text,       # thay context = joined_text
            "prompt": query_text,
            "response": response_text,
            "predict_label" : label 
          
        })
    except Exception as e:
        print(f"⚠️ Lỗi ở dòng {idx}: {e}")
        results.append({
            "id": row["id"],
            "context": input_text,                # lỗi thì context rỗng
            "prompt": query_text,
            "response": response_text,
            "predict_label": label
         
        })

# tạo DataFrame kết quả
df_results = pd.DataFrame(results)
df_results.to_csv("vihallu_train_with_joined.csv", index=False)

print("✅ Done! Đã lưu file vihallu_train_with_joined.csv")
print(df_results.head())


In [ ]:
df_results

In [ ]:
df_results.to_csv("Context_new.csv")